In [1]:
# This is code derived from online
# It currently works with no issue
# It must be run while the venv is active from the CLI

### SOURCE ###
# https://www.etedal.net/2020/04/pybullet-panda.html

import pybullet as p
import pybullet_data
import numpy as np
import h5py
import time
import os
import math
import tf

#DONT FORGET TO ACTIVATE VIRTUAL ENVIORNMENT: source pyvenv/bin/activate

pybullet build time: Nov 28 2023 23:51:11


### This is a function I defined to get the joint states at a given time

In [2]:
# Get current joint positions
def current_joint_positions(robot_id, joint_indices):
    current_joint_positions = []
    for joint_index in joint_indices:
        joint_state = p.getJointState(robot_id, joint_index)
        current_joint_positions.append(joint_state[0])  # Joint position is the first element of the tuple returned by getJointState
    
    return current_joint_positions

### Simloop

This loop runs for a specified number of simulations and spawns in a stack of block objects at a random position on the table and has the arm move towards the block's postition using inverse kinematics. The robot then grasps the object before stopping the simulation and saving the joint states of the robot during the simulation and saves them to a list of joint states from every simulation. I can make this a pandas dataframe too if it is more convinient. 

In [3]:
# Change this variable to change how many simulations are run

num_sims = 1

# Joint states from each simulation is published here
sim_results = []


for loop in range(num_sims):
    runsim = True

    p.connect(p.GUI)
    p.setGravity(0,0,-9.81)


    urdfRootPath=pybullet_data.getDataPath()
    kinova_uid = p.loadURDF('./robot/j2s6s300.urdf',useFixedBase=True)

    # Dictionary of Robot Active Joints and Home Position
    ARM_JOINTS = [2, 3, 4, 5, 6, 7]
    FINGER_JOINTS = [9, 10, 11, 12, 13, 14]

    #homePos = [0.0, 2.9, 1.3, 4.2, 1.4, 0.0, 0, 0, 0]
    homePos = [math.pi, math.pi, math.pi/2, math.pi, math.pi/2, math.pi]

    homeStateDict = dict(zip(ARM_JOINTS, homePos))

    # Sets robot home position before loading other objects
    for joint, pos in homeStateDict.items():
        p.resetJointState(kinova_uid, joint, pos)

    tableUid = p.loadURDF(os.path.join(urdfRootPath, "table/table.urdf"),basePosition=[0.5,0,-0.65])


    # Create a block collision shape
    blockShape = p.createCollisionShape(p.GEOM_BOX, halfExtents=[0.03, 0.03, 0.03])

    # Create a block rigid body
    blockBody1 = p.createMultiBody(baseMass=1.0, baseCollisionShapeIndex=blockShape)
    blockBody2 = p.createMultiBody(baseMass=1.0, baseCollisionShapeIndex=blockShape)

    initialOrientation = p.getQuaternionFromEuler([0, 0, 0])  # no rotation

    # Set the initial position and orientation of the block
    #initialPosition = [0.7, 0, 0.1]
    x_min, x_max = 0.1, 0.5  # Example limits for x coordinate
    y_min, y_max = -0.5, 0.5  # Example limits for y coordinate

    # Generate random x and y coordinates within the defined limits
    initial_x = np.random.uniform(x_min, x_max)
    initial_y = np.random.uniform(y_min, y_max)


    if np.random.uniform(0,1) > 0.5:
        initialPosition1 = [initial_x, initial_y, 0.1]
        initialPosition2 = [initial_x, initial_y, 0.1 + 0.03]

        p.resetBasePositionAndOrientation(blockBody1, initialPosition1, initialOrientation)
        p.resetBasePositionAndOrientation(blockBody2, initialPosition2, initialOrientation)

    else:
        initialPosition1 = [initial_x, initial_y, 0.1]
        initialPosition3 = [initial_x, initial_y, 0.1 + 0.03]
        initialPosition2 = [initial_x, initial_y, 0.1 + 0.03 + 0.03]

        blockBody3 = p.createMultiBody(baseMass=1.0, baseCollisionShapeIndex=blockShape)

        p.resetBasePositionAndOrientation(blockBody1, initialPosition1, initialOrientation)
        p.resetBasePositionAndOrientation(blockBody2, initialPosition2, initialOrientation)
        p.resetBasePositionAndOrientation(blockBody3, initialPosition3, initialOrientation)




    
    p.resetDebugVisualizerCamera(cameraDistance=1.5, cameraYaw=0, cameraPitch=-40, cameraTargetPosition=[0.55,-0.35,0.2])


    js_list = []


    state_durations = [0.25,1,0.25]
    control_dt = 1./240.
    p.setTimestep = control_dt
    state_t = 0.
    current_state = 0

    MAX_FINGER_POS = 1.5

    ikSolver = 0


    while runsim == True:
        p.stepSimulation()    
        
        js_list.append(current_joint_positions(kinova_uid, ARM_JOINTS))

        state_t += control_dt
        p.configureDebugVisualizer(p.COV_ENABLE_SINGLE_STEP_RENDERING) 
        
        if current_state == 0:
            # Sets robot home position before loading other objects
            for joint, pos in homeStateDict.items():
                p.setJointMotorControl2(kinova_uid, joint, p.POSITION_CONTROL, pos)


            
        if current_state == 1:
            # Moves arm towards object, I can change the orientation at which the arm grabs the object if needed
            objPos = p.getBasePositionAndOrientation(blockBody2)[0]

            handPos = [objPos[0], objPos[1], objPos[2]+0.01]
            
            roll = math.pi
            pitch = 0
            yaw = math.pi

            quaternion = tf.transformations.quaternion_from_euler(roll, pitch, yaw)
            
            handOr = [quaternion[0], quaternion[1], quaternion[2], quaternion[3]]


            jointPoses = p.calculateInverseKinematics(kinova_uid,
                                                            8,
                                                            handPos, handOr,
                                                            solver=ikSolver)

                
                
            for i in range(6):     
                
                p.setJointMotorControl2(bodyIndex=kinova_uid,
                                        jointIndex=i+2,
                                        controlMode=p.POSITION_CONTROL,
                                        targetPosition=jointPoses[i],
                                        targetVelocity=0,
                                        #force=500,
                                        positionGain=0.03,
                                        velocityGain=1
                                                        )
                                    

        if current_state == 2:
            # Closes the fingers to grab the object

            for i in FINGER_JOINTS:
                if i % 2 == 1:
                    p.setJointMotorControl2(kinova_uid, i,
                                        p.POSITION_CONTROL, 0.6 * MAX_FINGER_POS, force = 200)
                    p.setJointMotorControl2(kinova_uid, i + 1,
                                        p.POSITION_CONTROL, 0.5 * MAX_FINGER_POS, force = 200)

            
        

        if state_t >state_durations[current_state]:
            current_state += 1
            if current_state >= len(state_durations):
                sim_results.append(js_list) # Appends the joint states of arm during simulation at the sampling frequency control_dt
                p.disconnect()
                runsim = False
            state_t = 0

            

        


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Intel
GL_RENDERER=Mesa Intel(R) UHD Graphics (CML GT2)
GL_VERSION=4.6 (Core Profile) Mesa 21.2.6
GL_SHADING_LANGUAGE_VERSION=4.60
pthread_getconcurrency()=0
Version = 4.6 (Core Profile) Mesa 21.2.6
Vendor = Intel
Renderer = Mesa Intel(R) UHD Graphics (CML GT2)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf:

In [ ]:
print(sim_results)

[[[3.1415926535958203, 3.1415926536526193, 1.5707963244958318, 3.141592663776301, 1.5707963354334977, 3.1415926485142056], [3.14159265360125, 3.141592653709231, 1.5707963224261312, 3.1415926729527976, 1.5707963432060863, 3.1415926439409043], [3.1415926536001044, 3.1415926537599037, 1.5707963205535407, 3.14159268131698, 1.5707963502262212, 3.141592639787764], [3.1415926536063212, 3.1415926538059438, 1.570796318879465, 3.141592688733643, 1.5707963565171146, 3.1415926360836854], [3.1415926536116734, 3.141592653847347, 1.5707963173724657, 3.1415926954096562, 1.5707963621794268, 3.141592632750556], [3.141592653616296, 3.141592653884584, 1.570796316015901, 3.141592701418893, 1.570796367275916, 3.141592629751164], [3.1415926536203007, 3.1415926539180763, 1.5707963147947812, 3.141592706827864, 1.570796371863082, 3.1415926270520504], [3.1415926536237806, 3.1415926539482024, 1.570796313695604, 3.141592711696462, 1.570796375991791, 3.14159262462312], [3.141592653626813, 3.1415926539753025, 1.5707

### Gets the joint info for all joints in specified robot model

In [ ]:
# Get the number of joints in the robot
numJoints = p.getNumJoints(kinova_uid)

# Iterate over each joint to get its information
for jointIndex in range(numJoints):
    jointInfo = p.getJointInfo(kinova_uid, jointIndex)
    jointName = jointInfo[1].decode("utf-8")  # Decode the joint name
    print("Joint Index:", jointIndex, "Joint Name:", jointName)


error: Not connected to physics server.

# Debug Section

I used this section to see why the simulations would load but not run. It seems to be an issue with creating the point cloud, not sure what though

In [1]:
import pybullet as p
import pybullet_data
import os
import math
import numpy as np
from utils import current_joint_positions #, get_point_cloud
import h5py
import tf

# Configuration
NUM_SIMS = 10
SIM_RESULTS = []
ARM_JOINTS = [2, 3, 4, 5, 6, 7]
FINGER_JOINTS = [9, 10, 11, 12, 13, 14]
HOME_POS = [math.pi, math.pi, math.pi/2, math.pi, math.pi/2, math.pi] #[math.pi] * len(ARM_JOINTS); Straight up
STATE_DURATIONS = [1, 1, 1, 1, 1, 1 ] #[0.25, 1, 0.25, 0.25, 1, 0.25] 
CONTROL_DT = 1. / 2**9.
MAX_FINGER_POS = 1.5
GRASP_FORCE = 50
PID = [3 , 0.1, 0.1] # [Proportional Gain, Integral Gain, Derivative Gain]
MAX_VEL = 1

def setup_environment(with_gui):
    """
    Set up the PyBullet environment by loading the Kinova robot and a table.
    """
    urdf_root_path = pybullet_data.getDataPath()
    client_id = -1
    if with_gui:
        client_id = p.connect(p.GUI)
    else:
        client_id = p.connect(p.DIRECT)
    p.setGravity(0, 0, -9.81)
    p.setAdditionalSearchPath(urdf_root_path)
    p.resetDebugVisualizerCamera(3, 90, -30, [0.0, -0.0, -0.0])
    table_uid = p.loadURDF(os.path.join(urdf_root_path, "table/table.urdf"), basePosition=[0.5, 0, -0.65])
    kinova_uid = p.loadURDF("./robot/j2s6s300.urdf", useFixedBase=True)
    return kinova_uid, table_uid, client_id

def initialize_robot_position(kinova_uid):
    """
    Initialize the robot to the home position.
    """
    home_state_dict = dict(zip(ARM_JOINTS, HOME_POS))
    for joint, pos in home_state_dict.items():
        p.resetJointState(kinova_uid, joint, pos)

def create_object(num_object=1):
    """
    Create a block object in the environment.
    """
    block_shape = p.createCollisionShape(p.GEOM_BOX, halfExtents=[0.03, 0.03, 0.03])

    block_body = []
    for _ in range(num_object):
        block_body.append(p.createMultiBody(baseMass=1.0, baseCollisionShapeIndex=block_shape))
    
    x_min, x_max = 0.15, 0.55
    y_min, y_max = -0.45, 0.45
    initial_x = np.random.uniform(x_min, x_max)
    initial_y = np.random.uniform(y_min, y_max)

    initial_position = [initial_x, initial_y, 0.1]
    initial_orientation = p.getQuaternionFromEuler([0, 0, 0])
    
    for i in range(num_object):
        p.changeDynamics(block_body[i], -1, lateralFriction=10, spinningFriction=0) # Adding friction to the block object for grabbing
        p.resetBasePositionAndOrientation(block_body[i], initial_position, initial_orientation)

        initial_position[2] += 0.06


    return block_body

def run_simulation(kinova_uid, object_uid, sim_type, client_id):
    """
    Simulation loop that runs for a preset number of states depending on simulation type.
    """
    js_list = []
    current_state = 0
    state_time = 0.

    if sim_type == 1:
        x_min, x_max = 0.15, 0.55
        y_min, y_max = -0.45, 0.45
        initial_x = np.random.uniform(x_min, x_max)
        initial_y = np.random.uniform(y_min, y_max)

        initial_position = [initial_x, initial_y, 0.1]
        initial_orientation = p.getQuaternionFromEuler([0, 0, 0])

        p.resetBasePositionAndOrientation(object_uid[-1], initial_position, initial_orientation)

    with h5py.File("sim_results.h5", "w") as hdf_file:
        while True:
            p.stepSimulation()
            if current_state == 0:
                    #pc, mask = get_point_cloud()
                    #hdf_file.create_dataset('point_cloud', data=pc)
                    #hdf_file.create_dataset('mask', data=mask)
                    None
                  
            js_list.append(current_joint_positions(kinova_uid, ARM_JOINTS))
            #hdf_file.create_dataset('joint_states', data=js_list)


            if sim_type == 0:
                robot_reach(kinova_uid, object_uid[-1], current_state)

            elif sim_type == 1 and len(object_uid)>2:
                robot_stack_XonY(kinova_uid, object_uid_x=object_uid[-1], object_uid_y=object_uid[-2], state=current_state)
            else:
                print('Not enough items or sim_type is out of range')
                return
            
            state_time += CONTROL_DT
            if state_time > STATE_DURATIONS[current_state]:
                current_state += 1
                if current_state >= len(STATE_DURATIONS):
                    SIM_RESULTS.append(js_list)
                    break
                elif sim_type == 0 and current_state > 2:
                    SIM_RESULTS.append(js_list)
                    break
                state_time = 0
            

def robot_reach(kinova_uid, object_uid, state):
    """
    Robot control logic for each state for the reach task
    """
    if state == 1:
        object_position = p.getBasePositionAndOrientation(object_uid)[0]
        target_position = [object_position[0], object_position[1], object_position[2]+0.01]
            
        #Desired end effector orientation, Has the arm grab the block from above    
        roll = 0
        pitch = math.pi
        yaw = 0

        quaternion = tf.transformations.quaternion_from_euler(roll, pitch, yaw)
        target_orientation = [quaternion[0], quaternion[1], quaternion[2], quaternion[3]]

        current_positions = [p.getJointState(kinova_uid, i)[0] for i in ARM_JOINTS]
        joint_poses = p.calculateInverseKinematics(kinova_uid, 8, target_position, target_orientation, solver=0)
        position_error = [joint_poses[i] - current_positions[i] for i in range(len(current_positions))]
        targetVelocity = [i*PID[0] for i in position_error]

        for i in range(len(ARM_JOINTS)):
            p.setJointMotorControl2(kinova_uid, ARM_JOINTS[i], p.VELOCITY_CONTROL, targetVelocity = targetVelocity[i], maxVelocity = MAX_VEL)

    elif state == 2:
        for i in FINGER_JOINTS:
            if i % 2 == 1:
                p.setJointMotorControl2(kinova_uid, i,
                                    p.VELOCITY_CONTROL, targetVelocity = 2 ,force = GRASP_FORCE)
                p.setJointMotorControl2(kinova_uid, i + 1,
                                    p.VELOCITY_CONTROL, targetVelocity = 2 ,force = GRASP_FORCE)
                

def robot_stack_XonY(kinova_uid, object_uid_x, object_uid_y, state):
    """
    Robot control logic for each state for the stack skill
    """
    if state == 1:
        # Moves arm towards object
        objPos, objOr = p.getBasePositionAndOrientation(object_uid_x)


        handPos = [objPos[0], objPos[1], objPos[2]+0.015] #reaches right above top block
        handOr = p.getQuaternionFromEuler([0, -math.pi, 0]) # X, Y, Z

        # Get the current joint positions
        current_positions = [p.getJointState(kinova_uid, i)[0] for i in ARM_JOINTS]

        joint_poses = p.calculateInverseKinematics(kinova_uid, 8, handPos, handOr, solver=0)


        # The joint state is a tuple containing:
        # [0] - Joint position (in radians for revolute joints, in meters for prismatic joints)
        # [1] - Joint velocity
        # [2] - Joint reaction forces (torques for revolute joints, forces for prismatic joints)
        # [3] - Joint applied forces
            


        PosErr = [joint_poses[i] - current_positions[i] for i in range(len(current_positions))]

        targetVelocity = [i*PID[0] for i in PosErr]

        for i in range(len(ARM_JOINTS)):
            p.setJointMotorControl2(kinova_uid, ARM_JOINTS[i], p.VELOCITY_CONTROL, targetVelocity = targetVelocity[i], maxVelocity = MAX_VEL)


        
    if state == 2:
        for i in FINGER_JOINTS:
            if i % 2 == 1:
                p.setJointMotorControl2(kinova_uid, i,
                                    p.VELOCITY_CONTROL, targetVelocity = 2 ,force = GRASP_FORCE)
                p.setJointMotorControl2(kinova_uid, i + 1,
                                    p.VELOCITY_CONTROL, targetVelocity = 2 ,force = GRASP_FORCE)
                


    if state == 3:

        handPos = p.getLinkState(kinova_uid, 8)[0]

        handPos = [handPos[0], handPos[1], 0.3]


        handOr = p.getQuaternionFromEuler([0, -math.pi, 0]) # X, Y, Z

        # Get the current joint positions
        current_positions = [p.getJointState(kinova_uid, i)[0] for i in ARM_JOINTS]

        joint_poses = p.calculateInverseKinematics(kinova_uid,
                                                        8,
                                                        handPos, handOr,
                                                        solver=0)


        # The joint state is a tuple containing:
        # [0] - Joint position (in radians for revolute joints, in meters for prismatic joints)
        # [1] - Joint velocity
        # [2] - Joint reaction forces (torques for revolute joints, forces for prismatic joints)
        # [3] - Joint applied forces
            

        PosErr = [joint_poses[i] - current_positions[i] for i in range(len(current_positions))]

        targetVelocity = [i*PID[0] for i in PosErr]

        for i in range(len(ARM_JOINTS)):
            p.setJointMotorControl2(kinova_uid, ARM_JOINTS[i], p.VELOCITY_CONTROL, targetVelocity = targetVelocity[i], maxVelocity = MAX_VEL)

    if state == 4:
        
        # Moves arm towards object
        objPos, objOr = p.getBasePositionAndOrientation(object_uid_y)

        objInHandPos, _ = p.getBasePositionAndOrientation(object_uid_x)

        finalPos = [objPos[0], objPos[1], objPos[2]+0.1]

        # How far off center block is from being above the block
        blockPosErr = [finalPos[x] - objInHandPos[x] if abs(objPos[x] - objInHandPos[x]) < 0.1 else 0 for x in range(3)]

        # Should account for error if block isnt center in hand, Better PID tuning is needed for increased accuracy
        handPos = [objPos[0]+blockPosErr[0], objPos[1]+blockPosErr[1], objPos[2]+0.1+blockPosErr[2]] #places right on top block
        handOr = p.getQuaternionFromEuler([0, -math.pi, 0]) # X, Y, Z

        # Get the current joint positions
        current_positions = [p.getJointState(kinova_uid, i)[0] for i in ARM_JOINTS]

        joint_poses = p.calculateInverseKinematics(kinova_uid,
                                                        8,
                                                        handPos, handOr,
                                                        solver=0)


        PosErr = [joint_poses[i] - current_positions[i] for i in range(len(current_positions))]

        targetVelocity = [i*PID[0] for i in PosErr]

        for i in range(len(ARM_JOINTS)):
            p.setJointMotorControl2(kinova_uid, ARM_JOINTS[i], p.VELOCITY_CONTROL, targetVelocity = targetVelocity[i], maxVelocity = MAX_VEL)

    if state == 5:

        for i in FINGER_JOINTS:
            if i % 2 == 1:
                p.setJointMotorControl2(kinova_uid, i,
                                    p.POSITION_CONTROL, 0 * MAX_FINGER_POS)
                p.setJointMotorControl2(kinova_uid, i + 1,
                                    p.POSITION_CONTROL, 0 * MAX_FINGER_POS)


def simulate(num_sims, num_object, sim_type, with_gui=False):
    NUM_SIMS = num_sims
    for _ in range(NUM_SIMS):
        kinova_uid, _, client_id = setup_environment(with_gui)
        initialize_robot_position(kinova_uid)
        object_uid = create_object(num_object)
        run_simulation(kinova_uid, object_uid, sim_type, client_id)
        p.disconnect()



pybullet build time: Nov 28 2023 23:51:11


In [2]:
simulate(3, 1, 0, True)

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Intel
GL_RENDERER=Mesa Intel(R) UHD Graphics (CML GT2)
GL_VERSION=4.6 (Core Profile) Mesa 21.2.6
GL_SHADING_LANGUAGE_VERSION=4.60
pthread_getconcurrency()=0
Version = 4.6 (Core Profile) Mesa 21.2.6
Vendor = Intel
Renderer = Mesa Intel(R) UHD Graphics (CML GT2)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Intel
Workaround for some crash in the Intel OpenGL driver on Linux/Ubuntu
ven = Intel
Workaround for some crash in the Intel OpenGL driver on Linux/Ubuntu
b3Printf: b3Warning[examples/Importers/Imp